In [30]:

class WIN_STATUS:
    WIN = "STATUS_WIN"
    LOSS = "STATUS_LOSS"
    GAVANAM_THEVAI = "STATUS_GAVANAM_THEVAI"
    ATHIGA_GAVANAM_THEVAI = "STATUS_ATHIGA_GAVANAM_THEVAI"

def get_status(trend) -> str:
    status = None
    NUM_OF_YEARS = len(trend)

    loss_count = 0

    for i in trend:
        if i == "L":
            loss_count += 1

    if NUM_OF_YEARS == 2:
    # if years selected is 2
        if loss_count == 0:
            status = WIN_STATUS.WIN
        elif loss_count == 1:
            status = WIN_STATUS.GAVANAM_THEVAI
        else:
            status = WIN_STATUS.LOSS
    else:
        # if years selected are 3 or 4
        if loss_count == 0:
            status = WIN_STATUS.WIN
        elif loss_count == 1:
            status = WIN_STATUS.GAVANAM_THEVAI
        elif loss_count > 1 and loss_count < NUM_OF_YEARS:
            status = WIN_STATUS.ATHIGA_GAVANAM_THEVAI
        else:
            status = WIN_STATUS.LOSS

    return status

trend = "LW"

get_status(trend)

'STATUS_GAVANAM_THEVAI'